In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pymc as pm
import arviz as az

# Visual settings
sns.set_style("whitegrid")
plt.rcParams["figure.figsize"] = (12, 6)

%matplotlib inline

# Load Data
try:
    df = pd.read_csv('../data/BrentOilPrices_Processed.csv', index_col='Date', parse_dates=True)
    # If the processed file doesn't exist, try original
except FileNotFoundError:
    try:
        df = pd.read_csv('../data/BrentOilPrices.csv')
        df['Date'] = pd.to_datetime(df['Date'], format='%d-%b-%y')
        df.set_index('Date', inplace=True)
        df.sort_index(inplace=True)
    except FileNotFoundError:
         # Dummy generation again for safety
        print("Data not found. Generating dummy data.")
        dates = pd.date_range(start='1987-05-20', end='2022-09-30', freq='D')
        df = pd.DataFrame({
            'Price': np.concatenate([
                np.random.normal(20, 2, 5000),
                np.random.normal(50, 5, 4000),
                np.random.normal(100, 10, len(dates)-9000)
            ])
        }, index=dates)

# Filter for a specific range if needed to speed up calculation for demo (e.g., last 10 years)
# For full analysis, we use the whole dataset or specific periods of interest.
# df = df['2010':]

print(df.head())
plt.plot(df.index, df['Price'])
plt.title('Brent Oil Prices')
plt.show()

## 2. Load Event Data
Load the list of key geopolitical events to overlay on our results later.

In [ ]:
try:
    events_df = pd.read_csv('../data/events_template.csv')
    events_df['Date'] = pd.to_datetime(events_df['Date'])
    print("Events loaded:")
    print(events_df[['Date', 'Event']])
except FileNotFoundError:
    print("Events file not found.")
    events_df = pd.DataFrame(columns=['Date', 'Event'])

## 3. Bayesian Change Point Model with PyMC
We will model the price data assuming there is a single "switch point" ($\tau$) where the mean price changes from $\mu_1$ to $\mu_2$.

**Model Specification:**
1.  **Prior for $\tau$**: Discrete Uniform distribution over the time indices.
2.  **Priors for $\mu_1, \mu_2$**: Normal distributions (or Exponential if modeling positive values strictly).
3.  **Prior for $\sigma$**: Exponential distribution (for noise).
4.  **Likelihood**: Normal distribution $y \sim \mathcal{N}(\mu, \sigma)$ where $\mu$ switches at $\tau$.

*Note: For count data, Poisson is often used, but Price is continuous.*

In [ ]:
# Prepare data for PyMC
y_data = df['Price'].values
n_count = len(y_data)
idx = np.arange(n_count)

with pm.Model() as model:
    # 1. Define the Switch Point (tau)
    tau = pm.DiscreteUniform("tau", lower=0, upper=n_count - 1)

    # 2. Define Priors for Means
    # We can use the overall mean as a starting point
    mean_price = y_data.mean()
    mu_1 = pm.Normal("mu_1", mu=mean_price, sigma=20)
    mu_2 = pm.Normal("mu_2", mu=mean_price, sigma=20)

    # 3. Define Prior for Sigma
    sigma = pm.Exponential("sigma", lam=1.0)

    # 4. Define the Switch Function
    # pm.math.switch(condition, then, else)
    mu = pm.math.switch(tau >= idx, mu_1, mu_2)

    # 5. Define Likelihood
    observation = pm.Normal("obs", mu=mu, sigma=sigma, observed=y_data)

    # 6. Run the Sampler
    # Starting with a smaller number of samples for demonstration/speed
    trace = pm.sample(1000, tune=1000, return_inferencedata=True)

print("Sampling complete.")

## 4. Model Diagnostics
Check for convergence using `r_hat` and trace plots.
- `r_hat` should be close to 1.0 (< 1.05).
- Trace plots should look like "fuzzy caterpillars" (good mixing).

In [ ]:
# Summary Statistics
summary = az.summary(trace)
print(summary)

# Trace Plot
az.plot_trace(trace)
plt.tight_layout()
plt.show()

## 5. Visualizing the Change Point
Plot the posterior distribution of $\tau$ to see where the model thinks the change occurred.

In [ ]:
# Extract tau samples
tau_samples = trace.posterior['tau'].values.flatten()

# Plot
plt.figure(figsize=(15, 7))
plt.hist(tau_samples, bins=n_count, density=True, alpha=0.7, label='Posterior of $\\tau$')
plt.plot(df.index, df['Price'] / df['Price'].max(), alpha=0.5, label='Scaled Price')
plt.title('Posterior Distribution of Change Point vs Time')
plt.legend()
plt.show()

# Find the most probable change date(s)
# Note: tau corresponds to the index
tau_mode_idx = int(pd.DataFrame(tau_samples).mode().values[0])
change_date = df.index[tau_mode_idx]
print(f"Most likely change point index: {tau_mode_idx}")
print(f"Corresponding Date: {change_date.date()}")

## 6. Quantifying Impact
Compare the posterior distributions of $\mu_1$ (before) and $\mu_2$ (after).

In [ ]:
mu_1_samples = trace.posterior['mu_1'].values.flatten()
mu_2_samples = trace.posterior['mu_2'].values.flatten()

print(f"Mean Price Before: {mu_1_samples.mean():.2f} +/- {mu_1_samples.std():.2f}")
print(f"Mean Price After: {mu_2_samples.mean():.2f} +/- {mu_2_samples.std():.2f}")

az.plot_posterior(trace, var_names=['mu_1', 'mu_2'], ref_val=0)
plt.show()

# Calculate percentage change
pct_change_samples = (mu_2_samples - mu_1_samples) / mu_1_samples * 100
print(f"Average Percentage Change: {pct_change_samples.mean():.2f}%")

## 7. Event Association
Correlate the detected change point with known geopolitical events.

In [ ]:
# Find events near the change date (e.g., within 30 days)
window_days = 60
mask = (events_df['Date'] >= (change_date - pd.Timedelta(days=window_days))) & \
       (events_df['Date'] <= (change_date + pd.Timedelta(days=window_days)))
nearby_events = events_df[mask]

print(f"Events within {window_days} days of {change_date.date()}:")
if not nearby_events.empty:
    print(nearby_events)
else:
    print("No major events found in the filtered list within this window.")

# Visual check
plt.figure(figsize=(15, 7))
plt.plot(df.index, df['Price'], label='Price')
plt.axvline(x=change_date, color='red', linestyle='--', label=f'Detected Change: {change_date.date()}')

# Plot event markers
for _, row in events_df.iterrows():
    plt.axvline(x=row['Date'], color='green', alpha=0.3, linestyle=':')
    # Only label nearby events to avoid clutter
    if abs((row['Date'] - change_date).days) < 1000: # Show mostly relevant ones
         plt.text(row['Date'], df['Price'].max(), row['Event'], rotation=90, fontsize=8)

plt.title('Detected Change Point and Known Events')
plt.legend()
plt.show()